## Arranging Raw Images before Running SDFRED2

### Packages & Directories

In [1]:
import numpy as np
import glob, os
import pandas as pd
from pathlib import Path

In [2]:
dir_Raw = str(Path("./Raw").absolute())+"/"
dir_Flat = str(Path("./Flat").absolute())+"/"
dir_Obj = str(Path("./Object").absolute())+"/"

### Reading the data frame of raw images

In [3]:
# Loading data
df = pd.read_csv("raw_info.csv")
df.head(10)

,EXP-ID,FRAMEID,DATE-OBS,DATA-TYP,OBJECT,FILTER01,EXPTIME
0,SUPE01017660,SUPA01017660,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
1,SUPE01017660,SUPA01017661,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
2,SUPE01017660,SUPA01017662,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
3,SUPE01017660,SUPA01017663,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
4,SUPE01017660,SUPA01017664,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
5,SUPE01017660,SUPA01017665,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
6,SUPE01017660,SUPA01017666,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
7,SUPE01017660,SUPA01017667,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
8,SUPE01017660,SUPA01017668,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
9,SUPE01017660,SUPA01017669,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0


In [4]:
# Classification & counting
df.groupby(by=['DATA-TYP', 'FILTER01', 'DATE-OBS'], as_index=False).count()

,DATA-TYP,FILTER01,DATE-OBS,EXP-ID,FRAMEID,OBJECT,EXPTIME
0,DOMEFLAT,N-A-L656,2013-11-30,60,60,60,60
1,DOMEFLAT,N-A-L656,2013-12-03,140,140,140,140
2,DOMEFLAT,W-C-IC,2008-08-29,70,70,70,70
3,DOMEFLAT,W-C-IC,2008-08-30,70,70,70,70
4,DOMEFLAT,W-C-IC,2008-08-31,70,70,70,70
5,DOMEFLAT,W-C-IC,2013-12-02,50,50,50,50
6,DOMEFLAT,W-C-IC,2014-04-29,50,50,50,50
7,DOMEFLAT,W-C-RC,2009-08-22,10,10,10,10
8,DOMEFLAT,W-C-RC,2009-08-23,50,50,50,50
9,DOMEFLAT,W-C-RC,2013-12-01,60,60,60,60


### Arranging the Raw Images

In [5]:
df["FILTER01"].unique()

array(['W-C-IC', 'W-J-B', 'W-C-RC', 'N-A-L656'], dtype=object)

In [6]:
# For the flat, object paths
for root_path in [dir_Flat, dir_Obj]:
    for flt in df["FILTER01"].unique():
        if (glob.glob(root_path+flt) == []):
            os.system("mkdir "+root_path+flt)
            os.system("mkdir "+root_path+flt+"/Images")

In [7]:
# Checking the exposure time of object images
obj = (df["DATA-TYP"] == "OBJECT")
for flt in df["FILTER01"].unique():
    expt_series = df["EXPTIME"][obj & (df["FILTER01"] == flt)]
    expt_uniq = expt_series.unique()
    N_exp = expt_series.value_counts(sort=False).values
    tot_exp = 0
    message = flt+": "
    for NN in np.arange(len(N_exp)):
        tot_exp += expt_uniq[NN]*N_exp[NN]/10
        if (NN < np.arange(len(N_exp))[-1]):
            message += f"{expt_uniq[NN]:.1f} sec x {N_exp[NN]/10:.0f} + "
        else:
            message += f"{expt_uniq[NN]:.1f} sec x {N_exp[NN]/10:.0f} = {tot_exp:.1f} sec"
    print(message)

W-C-IC: 180.0 sec x 8 + 60.0 sec x 12 + 240.0 sec x 4 + 120.0 sec x 7 = 3960.0 sec
W-J-B: 240.0 sec x 7 + 200.0 sec x 1 + 60.0 sec x 8 + 180.0 sec x 9 = 3980.0 sec
W-C-RC: 560.0 sec x 1 + 60.0 sec x 4 + 240.0 sec x 3 + 120.0 sec x 9 = 2600.0 sec
N-A-L656: 120.0 sec x 5 + 480.0 sec x 9 = 4920.0 sec
